### Installing dependencies

In [4]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install datetime
!{sys.executable} -m pip install selenium
!{sys.executable} -m pip install nltk
!{sys.executable} -m pip install textblob
!{sys.executable} -m pip install lxml
!{sys.executable} -m pip install plotly

  Using cached https://files.pythonhosted.org/packages/10/ed/7e8b97591f6f456174139ec089c769f89a94a1a4025fe967691de971f314/bs4-0.0.1.tar.gz
  Stored in directory: C:\Users\Viraj\AppData\Local\pip\Cache\wheels\a0\b0\b2\4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
  Stored in directory: C:\Users\Viraj\AppData\Local\pip\Cache\wheels\d7\a9\33\acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying


# ---------------------------------------------------------------

# Main Code 
Will take atleast 10 mins

In [1]:
"""
This code was written by Viraj Shetty and Arjun Praveen!
"""
import pandas as pd
import time
import operator
import bs4 as bs
import datetime
from bs4 import BeautifulSoup
import urllib.request
from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from datetime import date
from textblob import TextBlob

"""
This little piece of code is added to make the webdriver run silently
Generally when you use WebDriver in Selenium it opens a new window
If you have automated some functions you can also see those functions happening live
To avoid it use this option called "headless" which does all this in the background

Web Driver object creation
Note: Before importing Selenium Webdriver
    Make sure you have a webdriver application installed in the same directory
    
    For Mozzilla
    >https://github.com/mozilla/geckodriver/releases
    
    For Chrome
    >https://chromedriver.chromium.org/downloads
"""
options = Options()
options.headless = True
browser = webdriver.Firefox(options = options)


# This function tokenizes the entered keywords

def tokenizer(l_t, l_d):
    v = TextBlob(l_t)
    w = TextBlob(l_d)
    a = v.noun_phrases
    b = w.noun_phrases
    c = a + b
    d = ""
    for i in c:
        d = d + i + " "
    print(d)
    return d

# Function for extracting number of days since the video has been uploaded

def number_of_days(v_date):
    today = date.today()
    month = {
             "Jan" : 1,
             "Feb" : 2,
             "Mar" : 3,
             "Apr" : 4,
             "May" : 5,
             "Jun" : 6,
             "Jul" : 7,
             "Aug" : 8,
             "Sep" : 9,
             "Oct" : 10,
             "Nov" : 11,
             "Dec" : 12
            }
    v=v_date.split(" ")
    v_year = int(v[2])
    v_month = int(month[v[0]])
    v_day = int(v[1])
    v_date = date(v_year,v_month,v_day)
    time = abs(today - v_date)
    return time.days

def search():
    
    keyword = input("Enter title: ")
    
    description = input("Enter description: ")  
    
    keysearch = tokenizer(keyword, description)
    keysearch_list = keysearch.split(" ")
    key = []
    for j in keysearch_list:
        if j not in key:
            key.append(j)
    print(key)
    keysearch = ""
    for k in key:
        keysearch = keysearch + k + " "
    print("Scraping search results for the keyword: " + keysearch)
    
    tag_list = keysearch.split(" ")
    browser = webdriver.Firefox()
    browser.get(f'https://www.youtube.com/results?search_query={keysearch}&sp=EgIQAQ%253D%253D')
    browser.maximize_window()

    """
    Please note here you need to use an Adblock to avoid advertisement videos to be a part of your list
    Add the filepath to your .xpi file
    """
    extension = "{your }" 
    #browser.install_addon(extension, temporary = True)

    # Change value of While Loop to scroll more pages
    i=0
    while i<=1:
        html = browser.find_element_by_tag_name('html')
        '''
        The code in the next line takes the page to end. Since in YouTube if you want more
        videos to load, you have to scroll down to the end of the page
        '''
        html.send_keys(Keys.END)
        time.sleep(3)
        i+=1
        
    v_l = browser.find_elements_by_xpath('//*[@id="video-title"]')
    link = []
    for i in v_l:
        link.append(i.get_attribute('href'))
        
    #print(link) 
    wait = WebDriverWait(browser,15)
    df = pd.DataFrame(columns = ['Link','Title','URL','Views','Likes','Dislikes','Subcount','Date','Days','Score'])
    k=0
    for x in link:
        try:
            i = 0
            browser.get(x)
            time.sleep(3)
            v_link = x
            v_title = wait.until(EC.presence_of_element_located((By.XPATH,'//h1[@class="title style-scope ytd-video-primary-info-renderer"]'))).text
            v_views = wait.until(EC.presence_of_element_located((By.XPATH,'/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[4]/div[1]/div/div[5]/div[2]/ytd-video-primary-info-renderer/div/div/div[1]/div[1]/yt-view-count-renderer/span[1]'))).text
            try:    
                v_views = int(v_views.strip(' views').replace(',', '', 2))
            except:
                v_views = 1
            v_likes = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'ytd-toggle-button-renderer.ytd-menu-renderer:nth-child(1) > a:nth-child(1) > yt-formatted-string:nth-child(2)'))).text
            v_dislikes = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'ytd-toggle-button-renderer.style-scope:nth-child(2) > a:nth-child(1) > yt-formatted-string:nth-child(2)'))).text
            
            #if (v_likes == 0 & v_dislikes == 0):
                #v_like_dislike = 0
            
            #Likes
            try:
                if (operator.contains(v_likes,'K')):
                    if(operator.contains(v_likes,'.')):
                        pos = len(v_likes)-v_likes.find('.')-1 #length from the end
                        a1 = v_likes.replace('.', '', 1)
                        num = (int(a1[:len(a1)-1])*1000)/(10**(pos-1))
                        v_likes = num
                    else:
                        v_likes=int(v_likes[ :len(v_likes)-1])*(1000)
                elif operator.contains(v_likes,'M'):
                    if(operator.contains(v_likes,'.')):
                        pos = len(v_likes)-v_likes.find('.')-1 #length from the end
                        a1 = v_likes.replace('.', '', 1)
                        num = (int(a1[:len(a1)-1])*1000000)/(10**(pos-1))
                        v_likes = num
                    else:
                        v_likes=int(v_likes[ :len(v_likes)-1])*(1000000)
                else:
                    v_likes = int(v_likes)
            except:
                v_likes = 1

            #Dislikes
            try:
                if (operator.contains(v_dislikes,'K')):
                    if(operator.contains(v_dislikes,'.')):
                        pos = len(v_dislikes)-v_dislikes.find('.')-1 #length from the end
                        a1 = v_dislikes.replace('.', '', 1)
                        num = (int(a1[:len(a1)-1])*1000)/(10**(pos-1))
                        v_dislikes = num
                    else:
                        v_dislikes=int(v_dislikes[ :len(v_dislikes)-1])*(1000)
                elif operator.contains(v_dislikes,'M'):
                    if(operator.contains(v_dislikes,'.')):
                        pos = len(v_dislikes)-v_dislikes.find('.')-1 #length from the end
                        a1 = v_dislikes.replace('.', '', 1)
                        num = (int(a1[:len(a1)-1])*1000000)/(10**(pos-1))
                        v_dislikes = num
                    else:
                        v_dislikes=int(v_dislikes[ :len(v_dislikes)-1])*(1000000)
                else:
                    v_dislikes = int(v_dislikes)
            except:
                v_dislikes = 1
            
            v_like_dislike = ((v_likes-v_dislikes)/v_likes)
            
            v_subcount = wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="owner-sub-count"]'))).text
            v_subcount = v_subcount.strip(" subscribers")
            try:
                if operator.contains(v_subcount,'M'):
                    if(operator.contains(v_subcount,'.')):
                        pos = len(v_subcount)-v_subcount.find('.')-1 #length from the end
                        a1 = v_subcount.replace('.', '', 1)
                        num = (int(a1[:len(a1)-1])*1000000)/(10**(pos-1))
                        v_subcount = num
                    else:
                        v_subcount=int(v_subcount[ :len(v_subcount)-1])*(1000000)
                elif operator.contains(v_subcount,'K'):
                    if(operator.contains(v_subcount,'.')):
                        pos = len(v_subcount)-v_subcount.find('.')-1 #length from the end
                        a1 = v_subcount.replace('.', '', 1)
                        num = (int(a1[:len(a1)-1])*1000)/(10**(pos-1))
                        v_subcount = num
                    else:
                        v_subcount = int(v_subcount[ :len(v_subcount)-1])*(1000)
                else:
                    v_subcount = int(float(v_subcount))
            except:
                v_subcount = 1
            v_urls = browser.find_elements_by_xpath('//div[@class="ytp-cued-thumbnail-overlay-image"]')
            for i in v_urls:
                v_url=i.get_attribute('style')[23:-3]
                break
            v_date = browser.find_element_by_xpath("//div[@id='date']").text
            if(operator.contains(v_date,'Premiered')):
                v_date = v_date[10:]
            elif(operator.contains(v_date,'Streamed live on')):
                v_date = v_date[17:]

            v_date = v_date[1:].replace(",","",1)
            v_days = number_of_days(v_date)
            url = x
            co =0

            
            # Score formula
            try:
                v_score = (((70*v_views)/700000) + (50*(v_likes/19000)) - (20*(v_dislikes/370)) + (160*v_like_dislike) +(20*(v_views/v_days)/7000)+(30*(v_likes/v_days)/200))/31
            except:
                v_days = 1
                v_likes = 1
                v_score = (((70*v_views)/700000) + (50*(v_likes/19000)) - (20*(v_dislikes/370)) + (160*v_like_dislike)+(20*(v_views/v_days)/7000)+(30*(v_likes/v_days)/200) )/31
            
            
            print('{}:  {:.2f}'.format(url, v_score))

            
            df.loc[len(df)] = [v_link,v_title,v_url,v_views,v_likes,v_dislikes,v_subcount,v_date,v_days,v_score]
            k
            k+=1
            print(k)
        #Exception added since some YouTube ads bypass the adblock
        except Exception as e:
            #print(e)
            continue
    #df = df.sort_values(by='Score', ascending = 0)
    return df
    browser.close()

n = int(input("How many Links would you like to print? "))
df1 = search()
df = df1.nlargest(n,'Score')
print(df)

How many Links would you like to print? 10
Enter title: Python
Enter description: Numpy
python numpy 
['python', 'numpy', '']
Scraping search results for the keyword: python numpy  
https://www.youtube.com/watch?v=8JfDAm9y_7s:  5.99
1
https://www.youtube.com/watch?v=QUT1VHiLmmI:  5.76
2
https://www.youtube.com/watch?v=xECXZ3tyONo:  5.24
3
https://www.youtube.com/watch?v=GB9ByFAIAH4:  6.26
4
https://www.youtube.com/watch?v=8LlXhtfNZEQ:  5.89
5
https://www.youtube.com/watch?v=Rbh1rieb3zc:  5.16
6
https://www.youtube.com/watch?v=ZB7BZMhfPgk:  5.35
7
https://www.youtube.com/watch?v=8Mpc9ukltVA:  5.52
8
https://www.youtube.com/watch?v=DI8wg3SRV90:  4.81
9
https://www.youtube.com/watch?v=pP5dQG0foTs:  4.83
10
https://www.youtube.com/watch?v=NYPKbmE0H6E:  5.78
11
https://www.youtube.com/watch?v=mOZ0UCeuRX4:  5.24
12
https://www.youtube.com/watch?v=jPEUXkW-DRE:  5.11
13
https://www.youtube.com/watch?v=oE4KeuVNqcQ:  5.36
14
https://www.youtube.com/watch?v=_d_Ka-ks2a0:  5.43
15
https://www.youtu

# ---------------------------------------------------------------

## Final Output

In [54]:
print("Top", n, "links: ")
print("------------------")
print(df[['Link', 'Title']])
df[['Link', 'Title']]

Top 10 links: 
------------------
                                           Link  \
31  https://www.youtube.com/watch?v=r-uOLxNrNk8   
3   https://www.youtube.com/watch?v=GB9ByFAIAH4   
0   https://www.youtube.com/watch?v=8JfDAm9y_7s   
4   https://www.youtube.com/watch?v=8LlXhtfNZEQ   
10  https://www.youtube.com/watch?v=NYPKbmE0H6E   
1   https://www.youtube.com/watch?v=QUT1VHiLmmI   
7   https://www.youtube.com/watch?v=8Mpc9ukltVA   
15  https://www.youtube.com/watch?v=a8aDcLk4vRc   
14  https://www.youtube.com/watch?v=_d_Ka-ks2a0   
54  https://www.youtube.com/watch?v=kbBi1liIrFg   

                                                Title  
31  Data Analysis with Python - Full Course for Be...  
3   Complete Python NumPy Tutorial (Creating Array...  
0   Python NumPy Tutorial | NumPy Array | Python T...  
4   #28 Python Tutorial for Beginners | Why Numpy?...  
10  #29 Python Tutorial for Beginners | Ways of Cr...  
1                 Python NumPy Tutorial for Beginners  
7    Python:

,Link,Title
31,https://www.youtube.com/watch?v=r-uOLxNrNk8,Data Analysis with Python - Full Course for Be...
3,https://www.youtube.com/watch?v=GB9ByFAIAH4,Complete Python NumPy Tutorial (Creating Array...
0,https://www.youtube.com/watch?v=8JfDAm9y_7s,Python NumPy Tutorial | NumPy Array | Python T...
4,https://www.youtube.com/watch?v=8LlXhtfNZEQ,#28 Python Tutorial for Beginners | Why Numpy?...
10,https://www.youtube.com/watch?v=NYPKbmE0H6E,#29 Python Tutorial for Beginners | Ways of Cr...
1,https://www.youtube.com/watch?v=QUT1VHiLmmI,Python NumPy Tutorial for Beginners
7,https://www.youtube.com/watch?v=8Mpc9ukltVA,Python: NUMPY | Numerical Python Arrays Tutorial
15,https://www.youtube.com/watch?v=a8aDcLk4vRc,numpy tutorial - basic array operations
14,https://www.youtube.com/watch?v=_d_Ka-ks2a0,"numpy tutorial - slicing/stacking arrays, inde..."
54,https://www.youtube.com/watch?v=kbBi1liIrFg,NumPy Linear Algebra - Dr. Ahmad Bazzi


# ---------------------------------------------------------------

# Plotting helper function

In [67]:
import plotly.graph_objects as go
def plotGraph(param, heading):
    df_new = df1.sort_values(by=param,ascending=0)
    df_l = df_new.head()
    list1 = []
    #This part
    for row in df_l.index: 
        list1.append(row)
    
    colors = ['lightslategray'] * len(df1)
    #This part
    for i in df1.index:
        if i in list1:
            colors[i] = 'green'
        else:
            colors[i] = 'grey'

    fig = go.Figure(data=[go.Bar(
        x=[i for i in range(len(df1))],
        y=[i for i in df1[param]],
        marker_color=colors # marker color can be a single color value or an iterable
    )])
    fig.update_layout(title_text=heading)
    fig.show()
# params = "Likes"
# df[params]


In [4]:

df1.sort_values(by='Score',ascending=0)

,Link,Title,URL,Views,Likes,Dislikes,Subcount,Date,Days,Score
31,https://www.youtube.com/watch?v=r-uOLxNrNk8,Data Analysis with Python - Full Course for Be...,https://i.ytimg.com/vi_webp/r-uOLxNrNk8/maxres...,96780,3600.0,36,1910000.0,Apr 15 2020,20,6.981660
3,https://www.youtube.com/watch?v=GB9ByFAIAH4,Complete Python NumPy Tutorial (Creating Array...,https://i.ytimg.com/vi_webp/IBiWRWJJE1E/maxres...,189730,6000.0,68,45400.0,Jul 10 2019,300,6.260659
0,https://www.youtube.com/watch?v=8JfDAm9y_7s,Python NumPy Tutorial | NumPy Array | Python T...,https://i.ytimg.com/vi_webp/8JfDAm9y_7s/maxres...,354670,3800.0,219,1660000.0,Apr 11 2017,1120,5.994252
4,https://www.youtube.com/watch?v=8LlXhtfNZEQ,#28 Python Tutorial for Beginners | Why Numpy?...,https://i.ytimg.com/vi_webp/IBiWRWJJE1E/maxres...,230952,2700.0,81,927000.0,Jul 20 2018,655,5.891865
10,https://www.youtube.com/watch?v=NYPKbmE0H6E,#29 Python Tutorial for Beginners | Ways of Cr...,https://i.ytimg.com/vi_webp/NYPKbmE0H6E/maxres...,184272,1900.0,40,927000.0,Jul 22 2018,653,5.778688
1,https://www.youtube.com/watch?v=QUT1VHiLmmI,Python NumPy Tutorial for Beginners,https://i.ytimg.com/vi_webp/QUT1VHiLmmI/maxres...,102161,2900.0,19,1910000.0,Aug 7 2019,272,5.756283
7,https://www.youtube.com/watch?v=8Mpc9ukltVA,Python: NUMPY | Numerical Python Arrays Tutorial,https://i.ytimg.com/vi_webp/8Mpc9ukltVA/maxres...,89764,1700.0,20,47900.0,Jun 10 2018,695,5.523309
15,https://www.youtube.com/watch?v=a8aDcLk4vRc,numpy tutorial - basic array operations,https://i.ytimg.com/vi_webp/a8aDcLk4vRc/maxres...,102579,1100.0,25,107000.0,Dec 26 2016,1226,5.436728
14,https://www.youtube.com/watch?v=_d_Ka-ks2a0,"numpy tutorial - slicing/stacking arrays, inde...",https://i.ytimg.com/vi_webp/_d_Ka-ks2a0/maxres...,68889,1400.0,16,107000.0,Dec 27 2016,1225,5.426186
54,https://www.youtube.com/watch?v=kbBi1liIrFg,NumPy Linear Algebra - Dr. Ahmad Bazzi,https://i.ytimg.com/vi_webp/kbBi1liIrFg/maxres...,19588,1700.0,4,28800.0,Sep 13 2018,600,5.366389


# ---------------------------------------------------------------

# Likes

In [75]:
plotGraph('Likes', 'Likes per video')

# ---------------------------------------------------------------

# Likes vs Dislikes


In [42]:
import plotly.graph_objects as go
vids = [x for x in range(25)]


fig = go.Figure(
    data=[
        go.Bar(name='Likes', x=vids, y=[df1.Likes[y] for y in range(25)], yaxis='y', marker_color='green', offsetgroup=1),
        go.Bar(name='Dislikes', x=vids, y=[df1.Dislikes[y] for y in range(25)], yaxis='y2', marker_color='red', offsetgroup=2)
    ],
    layout={
        'yaxis': {'title': 'Likes'},
        'yaxis2': {'title': 'Dislikes', 'overlaying': 'y', 'side': 'right'}
    }
)

# Change the bar mode
fig.update_layout(barmode='group')
fig.show()


##### Note: Click Likes/ Dislikes in the legend to isolate corresponding bar graphs
##### Note: Scale of both Y axis are different 


# ---------------------------------------------------------------

# Dislikes

In [72]:
plotGraph('Dislikes', 'Dislikes per video')

# ---------------------------------------------------------------

# Views

In [73]:
plotGraph('Views', 'Views per video')

# ---------------------------------------------------------------

# Subcount

In [74]:
plotGraph('Subcount', "Subcount per video")

In [43]:
# import plotly.graph_objects as go

# colors = ['lightslategray'] * len(df1)

# for i in df1.index:
#     if i in list1:
#         colors[i] = 'green'
#     else:
#         colors[i] = 'crimson'

# fig = go.Figure(data=[go.Bar(
#     x=[i for i in range(len(df1))],
#     y=[i for i in df1.Score],
#     marker_color=colors # marker color can be a single color value or an iterable
# )])
# fig.update_layout(title_text='Score per video')x